<a href="https://colab.research.google.com/github/SattuSupCodes/scrapper/blob/main/scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
!pip install google-generativeai

<frozen posixpath>:82: RuntimeWarning: coroutine 'scrape_company_site' was never awaited


In [39]:
import google.generativeai as genai
from getpass import getpass

GEMINI_API_KEY = getpass("Enter Gemini API Key: ")
genai.configure(api_key=GEMINI_API_KEY)

Enter Gemini API Key: ··········


In [40]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, urljoin
import re
import json

import google.generativeai as genai
from pprint import pprint


In [41]:
from google.colab import files

uploaded = files.upload()
file_name = list(uploaded.keys())[0]

with open(file_name, "r", encoding="utf-8") as f:
    raw_data = json.load(f)

pprint(raw_data.keys())


Saving copic_jp.json to copic_jp (2).json
dict_keys(['domain', 'source_url', 'scraped_at', 'title', 'meta_description', 'emails', 'socials', 'images', 'internal_pages'])


In [42]:
def prepare_ai_input(data):
    return {
        "domain": data.get("domain"),
        "title": data.get("title"),
        "meta_description": data.get("meta_description"),
        "emails": data.get("emails", [])[:3],
        "socials": data.get("socials", {}),
        "internal_pages": [
            {
                "url": p["url"],
                "title": p["title"],
                "headings": p["headings"]["h1"][:2],
                "text_sample": p["text_sample"][:800]
            }
            for p in data.get("internal_pages", [])[:3]
        ]
    }


In [43]:
ai_input = prepare_ai_input(raw_data)


In [52]:
def analyze_company_with_ai(clean_data):
    model = genai.GenerativeModel("gemini-3-flash-preview")

    prompt = f"""
You are analyzing a company's website.

Based ONLY on the information below, return a JSON object with:

- company_type (marketplace / SaaS / ecommerce / services / content / unclear)
- target_audience (B2B / B2C / both / unclear)
- trust_signals (list)
- risk_signals (list)
- transparency_level (low / medium / high)
- overall_confidence (low / medium / high)

Rules:
- Do NOT guess financials
- Do NOT invent facts
- If unclear, say "unclear"
- Output VALID JSON ONLY

DATA:
{json.dumps(clean_data, indent=2)}
"""

    response = model.generate_content(prompt)

    # Gemini sometimes wraps JSON in markdown — strip it safely
    text = response.text.strip()
    if text.startswith("```"):
        # More robust stripping of markdown and optional 'json' tag
        parts = text.split("```")
        if len(parts) >= 2: # Check if there's content after the first ```
            extracted_text = parts[1].strip()
            if extracted_text.startswith("json"): # Remove 'json' if present
                text = extracted_text[len("json"):].strip()
            else:
                text = extracted_text
        else:
            # Malformed markdown (e.g., only one ```), treat as empty
            text = ""

    # Attempt to parse JSON with error handling
    try:
        return json.loads(text)
    except json.JSONDecodeError as e:
        print(f"JSONDecodeError: {e}")
        print(f"Problematic text from model (length {len(text)}): '{text}'")
        # Return an empty dictionary or re-raise the error, depending on desired behavior
        return {} # Returning empty dict to avoid crashing the notebook

In [53]:
ai_analysis = analyze_company_with_ai(ai_input)
pprint(ai_analysis)


{'company_type': 'ecommerce',
 'overall_confidence': 'high',
 'risk_signals': ['No email addresses found in the provided content',
                  'No specific physical address or phone number listed in the '
                  'text'],
 'target_audience': 'both',
 'transparency_level': 'high',
 'trust_signals': ['Established brand founded in 1987',
                   'Official website of the Copic brand',
                   'Part of the Too Group in Tokyo, Japan',
                   'Comprehensive product catalog (358 colors, specific '
                   'product lines like Sketch, Ciao, Classic)',
                   'Presence of an official Instagram account',
                   'Detailed documentation on brand history and color systems']}
